In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//h1[@data-scroll-trigger-for-sticky-product-header]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                product_id = html.xpath('//div[@id="tabMatchingVehicles"]/@data-product-id')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_oe = [oe.strip() for oe in html.xpath('//div[@class="oe-number"]/descendant-or-self::*/text()')]

                while '' in list_oe:
                    list_oe.remove('')
                while ',' in list_oe:
                    list_oe.remove(',')

                oe = ';'.join(sorted(list(set(list_oe))))
                
                # = = = = = = = = = = = = = = =
                                
                list_div = html.xpath('//div[contains(@class, "description-block")]')
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(html.xpath('//script[@type="application/ld+json"]/text()')[0].strip())
                
                # = = = = = = = = = = = = = = =
                
                src = json_data['@graph'][0]['image'].strip()
                
                # = = = = = = = = = = = = = = =
                
                sku = json_data['@graph'][0]['sku'].strip()
                
                # = = = = = = = = = = = = = = =
                
                mpn = json_data['@graph'][0]['mpn'].strip()

                # = = = = = = = = = = = = = = =
                
                manufacturer = json_data['@graph'][0]['manufacturer'].strip()
                
                # = = = = = = = = = = = = = = =
                
                brand = json_data['@graph'][0]['brand']['name'].strip()
                
                # = = = = = = = = = = = = = = =
                
                price = json_data['@graph'][0]['offers'][0]['priceCurrency'].strip() + ' ' + json_data['@graph'][0]['offers'][0]['price'].strip()
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Sku': sku,
                                         'Brand': brand,
                                         'Manufacturer': manufacturer,
                                         'Mpn': mpn,
                                         'Product_Id': product_id,
                                         'Title': title,
                                         'Price': price,
                                         'Kit': input_.loc[a, 'Kit'],
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'OE': oe,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for div in list_div:
                    if 'delivery-contents' in div.xpath('./@class')[0]:
                        list_amount = [span.strip() for span in div.xpath('./div[@class="description-content"]/span[@class="delivery-content-count"]/text()')]
                        list_content = [content.strip() for content in div.xpath('./div[@class="description-content"]/text()')]
                        while '' in list_content:
                            list_content.remove('')

                        df_temp.loc[0, div.xpath('./div[@class="description-title"]/text()')[0].strip()] = '\n'.join([amount + ' ' + content for amount, content in zip(list_amount, list_content)])
                    elif 'technical-info' in div.xpath('./@class')[0]:
                        pass
                    else:
                        if len(div.xpath('./@style')) != 0 and div.xpath('./@style')[0].strip() != 'display: none;':
                            print('class: ' + div.xpath('./@class') + ', style: ' + div.xpath('./@style'))[0]

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Kit': input_.loc[a, 'Kit'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(17):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：150

[ok] - https://www.atp-autoteile.de/de/product/3648954-bestprice-bremsbelaege-vorne-satz
[尝试次数：1] - [剩余数量：133] - [当前时间：13:51:34]

[ok] - https://www.atp-autoteile.de/de/product/4091955-bestprice-8x-bremsbelaege-vwk-vorne-hinten
[尝试次数：1] - [剩余数量：132] - [当前时间：13:51:34]

[ok] - https://www.atp-autoteile.de/de/product/4316295-atec-satz-bremsbelaege-vwk-vorder-und-hinterachse
[尝试次数：1] - [剩余数量：131] - [当前时间：13:51:34]

[ok] - https://www.atp-autoteile.de/de/product/4234508-bestprice-8x-bremsbelaege-vorne-hinten
[尝试次数：1] - [剩余数量：130] - [当前时间：13:51:34]

[ok] - https://www.atp-autoteile.de/de/product/4234518-bestprice-8x-bremsbelaege-vorne-hinten
[尝试次数：1] - [剩余数量：129] - [当前时间：13:51:34]

[ok] - https://www.atp-autoteile.de/de/product/1144742-bestprice-bremsbelaege-vwk-vorderachse-satz
[尝试次数：1] - [剩余数量：128] - [当前时间：13:51:34]

[ok] - https://www.atp-autoteile.de/de/product/4234520-bestprice-8x-bremsbelaege-vorne-hinten
[尝试次数：1] - [剩余数量：127] - [当前时间：13:51:35]

[ok] - https://www.atp-autoteil

[ok] - https://www.atp-autoteile.de/de/product/1070229-bestprice-8x-bremsbelaege-vorder-und-hinterachse
[尝试次数：1] - [剩余数量：71] - [当前时间：13:51:49]

[ok] - https://www.atp-autoteile.de/de/product/4092407-bestprice-bremsbelaege-mit-vwk-vorne-satz
[尝试次数：1] - [剩余数量：70] - [当前时间：13:51:49]

[ok] - https://www.atp-autoteile.de/de/product/4203450-bestprice-bremsbelaege-vorne-satz
[尝试次数：1] - [剩余数量：69] - [当前时间：13:51:49]

[ok] - https://www.atp-autoteile.de/de/product/4405831-bestprice-8x-bremsbelaege-vorne-hinten
[尝试次数：1] - [剩余数量：68] - [当前时间：13:51:50]

[ok] - https://www.atp-autoteile.de/de/product/56656-bestprice-8x-bremsbelaege-vorder-hinterachse
[尝试次数：1] - [剩余数量：67] - [当前时间：13:51:50]

[ok] - https://www.atp-autoteile.de/de/product/4309328-atec-8x-bremsbelaege-warnkontakt-vorder-hinterachse
[尝试次数：1] - [剩余数量：66] - [当前时间：13:51:50]

[ok] - https://www.atp-autoteile.de/de/product/4305427-atec-8x-bremsbelaege-vorne-hinten
[尝试次数：1] - [剩余数量：65] - [当前时间：13:51:51]

[ok] - https://www.atp-autoteile.de/de/pro

[ok] - https://www.atp-autoteile.de/de/product/3649538-bestprice-bremsbelaege-hinten-satz
[尝试次数：1] - [剩余数量：9] - [当前时间：13:52:15]

[ok] - https://www.atp-autoteile.de/de/product/55546-atec-bremsbelaege-vorne-satz
[尝试次数：1] - [剩余数量：8] - [当前时间：13:52:16]

[ok] - https://www.atp-autoteile.de/de/product/4307431-atec-8x-bremsbelaege-ate-plastilube-vorne-hinten
[尝试次数：1] - [剩余数量：7] - [当前时间：13:52:16]

[ok] - https://www.atp-autoteile.de/de/product/3644574-bestprice-bremsbelaege-vorne-satz-mit-vwk
[尝试次数：1] - [剩余数量：6] - [当前时间：13:52:16]

[ok] - https://www.atp-autoteile.de/de/product/3644680-bestprice-bremsbelaege-vorne-satz
[尝试次数：1] - [剩余数量：5] - [当前时间：13:52:16]

[ok] - https://www.atp-autoteile.de/de/product/3631952-bestprice-bremsbelaege-vorne-satz-mit-vwk
[尝试次数：1] - [剩余数量：4] - [当前时间：13:52:17]

[ok] - https://www.atp-autoteile.de/de/product/44073-atec-bremsbelaege-vorne-satz-mit-vwk
[尝试次数：1] - [剩余数量：3] - [当前时间：13:52:17]

[ok] - https://www.atp-autoteile.de/de/product/48864-atec-bremsbelaege-vorne-s